# Frontend Development - Streamlit App

This notebook documents the Streamlit app development for the Cigna Claims Optimization System.

**Prerequisites:**
- **MUST run all previous notebooks first** (01-06)
- Complete dual-agent system working
- All agent functions and policy integration ready
- Streamlit in Snowflake enabled

**What this creates:**
- Streamlit app for provider interface (`streamlit_app.py`)
- Complete dual-agent workflow visualization
- Real-time claim generation and analysis
- Provider decision support interface
- Deployment configuration for Snowflake Streamlit

**GOAL:** Sales-ready demo showcasing Snowflake Cortex capabilities in healthcare claims optimization.


## Step 1: Verify System Readiness for Frontend

Confirm all backend components are ready for Streamlit integration.


In [ ]:
-- Frontend readiness verification
SELECT 
    'DATA_FOUNDATION' as COMPONENT_TYPE, 
    'PATIENTS' as COMPONENT,
    COUNT(*) as COUNT,
    'Cigna member data for demos' as PURPOSE
FROM CLAIMS_DEMO.PUBLIC.PATIENTS

UNION ALL

SELECT 'AGENT_SYSTEM', 'REGISTERED_AGENTS', COUNT(*), 'Builder + Insurance agents'
FROM CLAIMS_DEMO.PUBLIC.CORTEX_AGENTS_REGISTRY

UNION ALL

SELECT 'POLICY_SYSTEM', 'POLICY_DOCUMENTS', COUNT(*), 'Generated Cigna policies'
FROM CLAIMS_DEMO.PUBLIC.CIGNA_POLICY_DOCUMENTS

ORDER BY COMPONENT_TYPE;


In [ ]:
-- Verify agent functions for frontend integration
SELECT 
    function_name as AGENT_FUNCTION,
    argument_signature as INPUT_FORMAT,
    'READY_FOR_STREAMLIT' as STATUS
FROM CLAIMS_DEMO.INFORMATION_SCHEMA.FUNCTIONS 
WHERE function_schema = 'PUBLIC' 
  AND function_name IN ('BUILDER_AGENT', 'INSURANCE_AGENT_WITH_POLICY', 'BUILDER_AGENT_OPTIMIZE')
ORDER BY function_name;


## Step 2: Streamlit App Deployment Setup

Prepare for deploying the Streamlit app in Snowflake.


In [ ]:
-- Create stage for Streamlit app files
CREATE STAGE IF NOT EXISTS CLAIMS_DEMO.PUBLIC.STREAMLIT_STAGE;


## Step 3: Test App Components

Test individual components that the Streamlit app will use.


In [ ]:
-- Test complete workflow that Streamlit app will use
WITH provider_input AS (
    SELECT 
        'PAT_001' as PATIENT_ID,
        '85025' as PROCEDURE_CODE,
        'Annual wellness screening for diabetic patient' as CLINICAL_NOTES
),
builder_step AS (
    SELECT 
        *,
        CLAIMS_DEMO.PUBLIC.BUILDER_AGENT(PATIENT_ID, PROCEDURE_CODE, CLINICAL_NOTES) as GENERATED_CLAIM
    FROM provider_input
),
insurance_step AS (
    SELECT 
        *,
        CLAIMS_DEMO.PUBLIC.INSURANCE_AGENT_WITH_POLICY(GENERATED_CLAIM, PROCEDURE_CODE) as INSURANCE_ANALYSIS,
        TRY_PARSE_JSON(CLAIMS_DEMO.PUBLIC.INSURANCE_AGENT_WITH_POLICY(GENERATED_CLAIM, PROCEDURE_CODE)):strength_score::FLOAT as STRENGTH_SCORE
    FROM builder_step
)
SELECT 
    'STREAMLIT_WORKFLOW_TEST' as TEST_TYPE,
    PATIENT_ID,
    PROCEDURE_CODE,
    CASE WHEN GENERATED_CLAIM LIKE '%{%' THEN 'VALID_JSON' ELSE 'INVALID' END as BUILDER_OUTPUT,
    CASE WHEN INSURANCE_ANALYSIS LIKE '%{%' THEN 'VALID_JSON' ELSE 'INVALID' END as INSURANCE_OUTPUT,
    STRENGTH_SCORE,
    CASE WHEN STRENGTH_SCORE >= 0.7 THEN 'APPROVE' 
         WHEN STRENGTH_SCORE >= 0.4 THEN 'OPTIMIZE' 
         ELSE 'RECONSIDER' END as RECOMMENDATION
FROM insurance_step;


## Step 4: Deploy Streamlit App

Deploy the Cigna Claims Optimization app in Snowflake Streamlit.


In [ ]:
-- Deploy Streamlit app (after uploading files to stage)
CREATE OR REPLACE STREAMLIT CLAIMS_DEMO.PUBLIC.CIGNA_CLAIMS_OPTIMIZATION_APP
ROOT_LOCATION = '@CLAIMS_DEMO.PUBLIC.STREAMLIT_STAGE'
MAIN_FILE = 'streamlit_app.py'
QUERY_WAREHOUSE = 'DEMO_WH';


In [ ]:
-- Verify deployment and get app URL
SHOW STREAMLITS IN SCHEMA CLAIMS_DEMO.PUBLIC;

-- Get app details
DESC STREAMLIT CLAIMS_DEMO.PUBLIC.CIGNA_CLAIMS_OPTIMIZATION_APP;
